<a href="https://colab.research.google.com/github/BaoNguyen247/Side_project/blob/main/BirdStatistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About Dataset

This dataset contains 10,000 synthetic records simulating the migratory behavior of various bird species across global regions. Each entry represents a single bird tagged with a tracking device and includes detailed information such as flight distance, speed, altitude, weather conditions, tagging information, and migration outcomes.

The data was entirely synthetically generated using randomized yet realistic values based on known ranges from ornithological studies. It is ideal for practicing data analysis and visualization techniques without privacy concerns or real-world data access restrictions. Because it’s artificial, the dataset can be freely used in education, portfolio projects, demo dashboards, machine learning pipelines, or business intelligence training.

With over 40 columns, this dataset supports a wide array of analysis types. Analysts can explore questions like “Do certain species migrate in larger flocks?”, “How does weather impact nesting success?”, or “What conditions lead to migration interruptions?”. Users can also perform geospatial mapping of start and end locations, cluster birds by behavior, or build time series models based on migration months and environmental factors.

For data visualization, tools like Power BI, Python (Matplotlib/Seaborn/Plotly), or Excel can be used to create insightful dashboards and interactive charts.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Tiếp theo chúng ta đọc file csv

In [ ]:
df = pd.read_csv('/content/sample_data/bird_migration_data.csv')
#Hiển thị dữ liệu
print(df.head())

Bước đầu tiên chúng ta hãy cùng xem xét lại data, check xem data có sai hay không?

In [ ]:
# Kiểm tra giá trị thiếu trong Interrupted_Reason
missing_reason = df[df['Interrupted_Reason'].isnull()]
print("Số lượng giá trị thiếu trong Interrupted_Reason:", len(missing_reason))
print("Phân bố Migration_Interrupted trong các giá trị thiếu:")
print(missing_reason['Migration_Interrupted'].value_counts())

Ta thấy ở cột interrupted_Reason có 1981 giá trị bị thiếu, điều này nói lên rằng có những yếu tố làm gián đoạn quá trình di cư của loài chim không được đề cập ở đây, có thể là do không theo dõi sát sao trong quá trình di cư, tuy nhiên những dữ liệu của các cột còn lại rất sạch nên những dữ liệu bị thiếu sẽ không ảnh hưởng nhiều đến quá trình phân tích của chúng ta, bây giờ chúng ta hãy xem những mẫu dữ liệu này nói lên điều gì từ loài chim nhé.

In [ ]:
df['Interrupted_Reason'].fillna('None', inplace=True)

# 1.Đầu tiên ta xem xét về vấn đề quãng thời gian chúng di cư và lộ trình của các loài chim

In [ ]:
# Lọc các bản ghi có latitude/longitude hợp lệ
df = df[
    (df['Start_Latitude'].between(-90, 90)) &
    (df['End_Latitude'].between(-90, 90)) &
    (df['Start_Longitude'].between(-180, 180)) &
    (df['End_Longitude'].between(-180, 180))
]
print("Số bản ghi sau khi lọc:", len(df))

In [ ]:
# Kiểm tra thêm giá trị bất thường như quãng đường bay, v.v,...
print("Flight_Distance_km âm:", (df['Flight_Distance_km'] < 0).sum())
print("Flight_Duration_hours âm:", (df['Flight_Duration_hours'] < 0).sum())
print("Average_Speed_kmph âm:", (df['Average_Speed_kmph'] < 0).sum())
print("Min_Altitude_m > Max_Altitude_m:", (df['Min_Altitude_m'] > df['Max_Altitude_m']).sum())

In [ ]:
# Làm sạch dữ liệu
df = df[
    (df['Start_Latitude'].between(-90, 90)) &
    (df['End_Latitude'].between(-90, 90)) &
    (df['Start_Longitude'].between(-180, 180)) &
    (df['End_Longitude'].between(-180, 180)) &
    (df['Flight_Distance_km'] >= 0) &
    (df['Flight_Duration_hours'] >= 0) &
    (df['Average_Speed_kmph'] >= 0) &
    (df['Min_Altitude_m'] <= df['Max_Altitude_m'])
]
print("Số bản ghi sau khi làm sạch:", len(df))

In [ ]:
#Chúng ta thống kê xem hiện có bao nhiêu loài chim có trong danh sách
new_df_unit = df['Species'].drop_duplicates()
print(new_df_unit)

ta thấy có 7 chủng loài chim, tuy nhiên mỗi vùng và mỗi khu vực các loài chim này có những đặc điểm sống khác nhau, hãy xem xét về loài Warbler(chim chích)ở các khu vực trên thế giới

Từ dữ liệu mẫu bạn cung cấp, có bản ghi với Start_Latitude = 87.861164 (hợp lệ) nhưng cũng có Start_Latitude = -169.378251 (không hợp lệ, vì latitude chỉ nằm trong [-90, 90]). Điều này cho thấy dữ liệu giả lập có thể chứa các giá trị sai, cần được làm sạch trước khi phân tích.

In [ ]:
warbler_df = df[df['Species'] == "Warbler"]
print(warbler_df.head())

Hãy xem môi trường sống có ảnh hưởng như thế nào đến lý do di cư của chúng

In [ ]:
# Đếm số lần xuất hiện của từng lý do di cư trong mỗi môi trường sống
data = warbler_df.groupby("Habitat")["Migration_Reason"].value_counts().unstack()

In [ ]:
# Thiết lập vị trí các cột
x = np.arange(len(data.index))  # Số lượng môi trường sống
width = 0.2  # Độ rộng mỗi cột

# Vẽ biểu đồ
fig, ax = plt.subplots(figsize=(12, 6))

for i, reason in enumerate(data.columns):
    ax.bar(x + i * width, data[reason], width, label=reason)

# Tùy chỉnh
ax.set_xlabel("Môi trường sống")
ax.set_ylabel("Số lượng")
ax.set_title("Ảnh hưởng của môi trường sống đến lý do di cư của loài chim chích")
ax.set_xticks(x + width * (len(data.columns) / 2))
ax.set_xticklabels(data.index, rotation=45)
ax.legend(title="Lý do di cư")

# Hiển thị
plt.show()


Ta thấy xu hướng di cư để né tránh thiên địch có tỉ lệ cao ở Coastal và Urban khả năn là do các loài thiên địch xu hướng hoạt động mạnh mẽ hơn vào mùa loài chim này di cư, nhóm ở Mountain và Wetland xu hướng di cư dể tìm thức ăn, chứng tỏ ở 2 khu vực này có hiện tượng thiếu thống thức ăn ở mức nhẹ vào mùa di cư
, trong khi các khu vực còn lại có xu hướng cân bằng lí do di cư

In [ ]:
hawk_df = df[df['Species'] == "Hawk"]
data = hawk_df.groupby("Habitat")["Migration_Reason"].value_counts().unstack()
# Thiết lập vị trí các cột
x = np.arange(len(data.index))  # Số lượng môi trường sống
width = 0.2  # Độ rộng mỗi cột

# Vẽ biểu đồ
fig, ax = plt.subplots(figsize=(12, 6))

for i, reason in enumerate(data.columns):
    ax.bar(x + i * width, data[reason], width, label=reason)

# Tùy chỉnh
ax.set_xlabel("Môi trường sống")
ax.set_ylabel("Số lượng")
ax.set_title("Ảnh hưởng của môi trường sống đến lý do di cư của loài Diều hâu")
ax.set_xticks(x + width * (len(data.columns) / 2))
ax.set_xticklabels(data.index, rotation=45)
ax.legend(title="Lý do di cư")

# Hiển thị
plt.show()


In [ ]:
eagle_df = df[df['Species'] == "Eagle"]
data = eagle_df.groupby("Habitat")["Migration_Reason"].value_counts().unstack()
# Thiết lập vị trí các cột
x = np.arange(len(data.index))  # Số lượng môi trường sống
width = 0.2  # Độ rộng mỗi cột

# Vẽ biểu đồ
fig, ax = plt.subplots(figsize=(12, 6))

for i, reason in enumerate(data.columns):
    ax.bar(x + i * width, data[reason], width, label=reason)

# Tùy chỉnh
ax.set_xlabel("Môi trường sống")
ax.set_ylabel("Số lượng")
ax.set_title("Ảnh hưởng của môi trường sống đến lý do di cư của loài Đại bàng")
ax.set_xticks(x + width * (len(data.columns) / 2))
ax.set_xticklabels(data.index, rotation=45)
ax.legend(title="Lý do di cư")

# Hiển thị
plt.show()

Phân tích thời điểm di cư

Cột liên quan:

    Migration_Start_Month: Tháng bắt đầu di cư (giá trị từ 1 đến 12).
    Migration_End_Month: Tháng kết thúc di cư (giá trị từ 1 đến 12).
    Species: Loài chim (Warbler, Stork, Hawk, Crane, Goose, Eagle, Swallow).
    Region: Khu vực địa lý (South America, North America, Asia, Europe, v.v.).
    Habitat: Môi trường sống (Grassland, Urban, Wetland, Mountain, Coastal, v.v.).
    Weather_Condition, Temperature_C: Điều kiện thời tiết và nhiệt độ, có thể ảnh hưởng đến thời điểm di cư.

    

In [ ]:
# Đọc dữ liệu
df = pd.read_csv('/content/sample_data/bird_migration_data.csv')

# Làm sạch dữ liệu
# Sửa giá trị thiếu cho Interrupted_Reason
df['Interrupted_Reason'] = df['Interrupted_Reason'].fillna('None')

# Lọc các bản ghi có latitude/longitude hợp lệ (từ bước trước)
df = df[
    (df['Start_Latitude'].between(-90, 90)) &
    (df['End_Latitude'].between(-90, 90)) &
    (df['Start_Longitude'].between(-180, 180)) &
    (df['End_Longitude'].between(-180, 180)) &
    (df['Flight_Distance_km'] >= 0) &
    (df['Flight_Duration_hours'] >= 0) &
    (df['Average_Speed_kmph'] >= 0) &
    (df['Min_Altitude_m'] <= df['Max_Altitude_m'])
]
print("Số bản ghi sau khi làm sạch:", len(df))

# Phân tích thời điểm di cư theo loài
# Tính tần suất Migration_Start_Month theo Species
start_month_by_species = df.groupby('Species')['Migration_Start_Month'].value_counts().unstack().fillna(0)

# Tính tần suất Migration_End_Month theo Species
end_month_by_species = df.groupby('Species')['Migration_End_Month'].value_counts().unstack().fillna(0)

# In kết quả
print("Tần suất Migration_Start_Month theo loài:\n", start_month_by_species)
print("Tần suất Migration_End_Month theo loài:\n", end_month_by_species)

# Trực quan hóa Migration_Start_Month
plt.figure(figsize=(14, 8))
sns.heatmap(start_month_by_species, cmap='Blues', annot=True, fmt='.0f')
plt.title("Phân bố tháng bắt đầu di cư theo loài chim")
plt.xlabel("Tháng")
plt.ylabel("Loài chim")
plt.show()

# Trực quan hóa Migration_End_Month
plt.figure(figsize=(14, 8))
sns.heatmap(end_month_by_species, cmap='Oranges', annot=True, fmt='.0f')
plt.title("Phân bố tháng kết thúc di cư theo loài chim")
plt.xlabel("Tháng")
plt.ylabel("Loài chim")
plt.show()

# Phân tích theo Region (tùy chọn)
start_month_by_region = df.groupby('Region')['Migration_Start_Month'].value_counts().unstack().fillna(0)
plt.figure(figsize=(14, 8))
sns.heatmap(start_month_by_region, cmap='Greens', annot=True, fmt='.0f')
plt.title("Phân bố tháng bắt đầu di cư theo khu vực")
plt.xlabel("Tháng")
plt.ylabel("Khu vực")
plt.show()

# Phân tích ảnh hưởng của nhiệt độ đến thời điểm di cư
plt.figure(figsize=(10, 6))
sns.boxplot(x='Migration_Start_Month', y='Temperature_C', data=df)
plt.title("Nhiệt độ theo tháng bắt đầu di cư")
plt.xlabel("Tháng bắt đầu di cư")
plt.ylabel("Nhiệt độ (°C)")
plt.show()

# Phân tích ảnh hưởng của thời tiết đến thời điểm di cư
weather_start_month = df.groupby('Weather_Condition')['Migration_Start_Month'].value_counts().unstack().fillna(0)
plt.figure(figsize=(14, 8))
sns.heatmap(weather_start_month, cmap='Purples', annot=True, fmt='.0f')
plt.title("Phân bố tháng bắt đầu di cư theo tình trạng thời tiết")
plt.xlabel("Tháng")
plt.ylabel("Tình trạng thời tiết")
plt.show()

1. Phân tích biểu đồ 1: "Phân bố tháng bắt đầu di cư theo loài chim"

    Mô tả: Biểu đồ heatmap cho thấy tần suất Migration_Start_Month (tháng bắt đầu di cư) theo các loài chim (Warbler, Swallow, Stork, Hawk, Goose, Eagle, Crane) qua các tháng (Jan, Feb, Mar, Apr, May, Sep, Oct, Nov).
    Phân tích:
        Warbler: Đỉnh cao di cư vào tháng 3 (202) và tháng 11 (196), cho thấy hai mùa di cư chính (mùa xuân và mùa thu). Điều này có thể liên quan đến việc tìm thức ăn hoặc tránh mùa đông.
        Swallow: Đỉnh cao vào tháng 3 (228) và tháng 11 (217), tương tự Warbler, có thể phản ánh hành vi di cư dài để tìm vùng ấm áp hoặc sinh sản.
        Stork: Đỉnh cao vào tháng 3 (180) và tháng 4 (186), phù hợp với mùa sinh sản ở vùng ôn đới.
        Hawk: Đỉnh cao rõ rệt vào tháng 3 (205), với xu hướng di cư mùa xuân, có thể để tránh thiên địch hoặc tìm thức ăn.
        Goose: Đỉnh cao vào tháng 3 (191) và tháng 11 (179), cho thấy di cư hai chiều (xuân và thu) để sinh sản và tránh mùa đông.
        Eagle: Đỉnh cao vào tháng 3 (172) và tháng 11 (184), tương tự các loài khác, nhưng ít biến động hơn.
        Crane: Đỉnh cao vào tháng 3 (175) và tháng 4 (176), phù hợp với di cư mùa xuân để sinh sản.
2. Kết luận
  Tháng 3 là thời điểm di cư phổ biến nhất cho hầu hết các loài (trừ Warbler và Swallow có thêm đỉnh tháng 11), phản ánh xu hướng di cư mùa xuân để sinh sản hoặc tìm thức ăn.
  Các loài như Warbler, Swallow, và Goose có xu hướng di cư hai mùa (xuân và thu), liên quan đến mục đích tránh điều kiện bất lợi (mùa đông) và tìm nguồn thức ăn.


Phân tích biểu đồ 2: "Phân bố tháng kết thúc di cư theo loài chim"

    Mô tả: Biểu đồ heatmap cho thấy tần suất Migration_End_Month (tháng kết thúc di cư) theo các loài chim qua các tháng (Dec, Jan, Mar, Apr, May, Jun, Nov, Oct).
    Phân tích:
        Warbler: Đỉnh cao vào tháng 3 (228) và tháng 11 (217), tương tự tháng bắt đầu, cho thấy hành trình di cư kéo dài qua mùa xuân và thu.
        Swallow: Đỉnh cao vào tháng 3 (205) và tháng 11 (213), nhất quán với mùa xuân và thu.
        Stork: Đỉnh cao vào tháng 4 (224) và tháng 5 (210), phản ánh kết thúc di cư sau mùa sinh sản.
        Hawk: Đỉnh cao rõ rệt vào tháng 4 (221) và tháng 5 (193), phù hợp với kết thúc di cư mùa xuân.
        Goose: Đỉnh cao vào tháng 4 (205) và tháng 11 (220), cho thấy di cư kết thúc ở cả mùa xuân và thu.
        Eagle: Đỉnh cao vào tháng 4 (200) và tháng 11 (216), tương tự Goose.
        Crane: Đỉnh cao vào tháng 4 (203) và tháng 5 (202), nhất quán với mùa sinh sản.
    Kết luận:
        Tháng 4 và tháng 5 là thời điểm kết thúc di cư phổ biến, đặc biệt cho các loài như Stork, Hawk, và Crane, liên quan đến việc đến khu vực sinh sản hoặc ổn định sau di cư.
        Warbler và Swallow có xu hướng kết thúc di cư vào tháng 11, có thể do di cư dài đến vùng nhiệt đới để tránh mùa đông.

3. Phân tích biểu đồ 3: "Phân bố tháng bắt đầu di cư theo khu vực"

    Mô tả: Biểu đồ heatmap cho thấy tần suất Migration_Start_Month theo các khu vực (South America, North America, Europe, Australia, Asia, Africa) qua các tháng (Feb, Jan, Mar, May, Nov, Oct, Sep).
    Phân tích:
        South America: Đỉnh cao vào tháng 9 (242), có thể liên quan đến di cư theo mùa khô hoặc mùa sinh sản ở Nam bán cầu.
        North America: Đỉnh cao vào tháng 3 (226) và tháng 9 (218), phản ánh di cư xuân và thu để tránh mùa đông hoặc tìm thức ăn.
        Europe: Đỉnh cao vào tháng 3 (208) và tháng 9 (214), tương tự North America, do khí hậu ôn đới.
        Australia: Đỉnh cao vào tháng 3 (228), có thể liên quan đến mùa sinh sản hoặc di cư nội địa.
        Asia: Đỉnh cao vào tháng 3 (204) và tháng 9 (222), phản ánh di cư hai mùa.
        Africa: Đỉnh cao vào tháng 3 (239), có thể do di cư từ các vùng ôn đới đến khu vực nhiệt đới.
    Kết luận:
        Tháng 3 là thời điểm di cư phổ biến nhất trên toàn cầu, đặc biệt ở North America, Europe, và Africa, liên quan đến mùa xuân.
        South America có xu hướng di cư vào tháng 9, phản ánh sự khác biệt giữa Nam và Bắc bán cầu.

    Phân tích biểu đồ 4: "Nhiệt độ theo tháng bắt đầu di cư"

    Mô tả: Biểu đồ boxplot cho thấy phân bố Temperature_C (nhiệt độ) theo Migration_Start_Month (Jan, Apr, May, Oct, Nov, Feb, Mar, Sep).
    Phân tích:
        Jan: Nhiệt độ trung bình khoảng 0°C, với phạm vi từ -10°C đến 10°C, phản ánh điều kiện lạnh thúc đẩy di cư tránh mùa đông.
        Feb: Nhiệt độ trung bình khoảng 5°C, với phạm vi từ -5°C đến 15°C, cho thấy sự ấm dần.
        Mar: Nhiệt độ trung bình khoảng 10°C, với phạm vi từ 0°C đến 20°C, phù hợp với mùa xuân.
        Apr: Nhiệt độ trung bình khoảng 15°C, với phạm vi từ 5°C đến 25°C, lý tưởng cho sinh sản.
        May: Nhiệt độ trung bình khoảng 20°C, với phạm vi từ 10°C đến 30°C, phản ánh điều kiện ổn định.
        Sep: Nhiệt độ trung bình khoảng 15°C, với phạm vi từ 5°C đến 25°C, liên quan đến mùa thu.
        Oct: Nhiệt độ trung bình khoảng 10°C, với phạm vi từ 0°C đến 20°C, cho thấy sự lạnh dần.
        Nov: Nhiệt độ trung bình khoảng 5°C, với phạm vi từ -5°C đến 15°C, thúc đẩy di cư tránh mùa đông.
    Kết luận:
        Nhiệt độ tăng từ tháng 1 đến tháng 5 (0°C đến 20°C) và giảm từ tháng 9 đến tháng 11 (15°C đến 5°C), phản ánh nhịp sinh học theo mùa.
        Di cư thường bắt đầu khi nhiệt độ đạt khoảng 10-15°C (tháng 3-4), phù hợp với sinh sản hoặc tìm thức ăn.

5. Phân tích biểu đồ 5: "Phân bố tháng bắt đầu di cư theo tình trạng thời tiết"

    Mô tả: Biểu đồ heatmap cho thấy tần suất Migration_Start_Month theo Weather_Condition (Clear, Foggy, Rainy, Stormy, Windy) qua các tháng (Apr, Feb, Jan, Mar, May, Nov, Oct, Sep).
    Phân tích:
        Clear: Đỉnh cao vào tháng 3 (240) và tháng 11 (267), cho thấy chim ưu tiên di cư trong điều kiện thời tiết trong lành.
        Foggy: Đỉnh cao vào tháng 3 (292), nhưng ít phổ biến hơn, có thể do tầm nhìn hạn chế.
        Rainy: Đỉnh cao vào tháng 3 (248) và tháng 11 (264), cho thấy di cư trong mưa là khả thi nhưng không tối ưu.
        Stormy: Đỉnh cao vào tháng 3 (270) và tháng 11 (283), nhưng tần suất thấp hơn, phản ánh tránh thời tiết xấu.
        Windy: Đỉnh cao vào tháng 3 (263) và tháng 11 (287), có thể tận dụng gió để di cư.
    Kết luận:
        Tháng 3 và tháng 11 là thời điểm di cư phổ biến bất kể thời tiết, nhưng điều kiện Clear được ưu tiên.
        Stormy và Foggy có tần suất thấp, cho thấy chim tránh di cư trong điều kiện bất lợi.

6. Tổng kết và liên hệ với yếu tố quyết định di cư
Yếu tố quyết định di cư:

    Nhịp sinh học: Các loài di cư tập trung vào tháng 3 (xuân) và tháng 11 (thu), phản ánh đồng hồ sinh học điều chỉnh theo độ dài ngày và mùa.
    Điều kiện môi trường:
        Nhiệt độ: Di cư tăng khi nhiệt độ đạt 10-15°C (tháng 3-4), giảm khi nhiệt độ xuống dưới 5°C (tháng 11-1).
        Thời tiết: Chim ưu tiên Clear và Windy, tránh Stormy và Foggy, cho thấy chiến lược tránh rủi ro.
    Khoảng cách địa lý: South America có đỉnh di cư tháng 9 (Nam bán cầu), trong khi North America và Europe di cư tháng 3 (Bắc bán cầu), phản ánh sự khác biệt theo mùa.
    Đặc điểm sinh học: Các loài như Warbler và Swallow di cư hai mùa, trong khi Hawk và Crane tập trung vào mùa xuân, liên quan đến chiến lược sinh sản hoặc săn mồi.

Mục đích di cư:

    Sinh sản: Tháng 3-5 (đỉnh tháng 4) là thời điểm kết thúc di cư, phù hợp cho Stork, Crane, và Hawk để đến khu vực sinh sản.
    Tìm thức ăn: Warbler và Swallow di cư vào tháng 11, có thể để tìm thức ăn ở vùng nhiệt đới.
    Tránh điều kiện bất lợi: Di cư tháng 9-11 (North America, Europe) phản ánh việc tránh mùa đông lạnh.

# 2.Phân tích lộ trình và khoảng cách di cư

In [ ]:
# Lọc dữ liệu chỉ gồm 3 loài chim
species_list = ['Warbler', 'Hawk', 'Eagle']
df_filtered = df[df['Species'].isin(species_list)]

In [ ]:
# Tính khoảng cách di cư (Haversine)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2.0)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

df_filtered['Flight_Distance_km'] = haversine(
    df_filtered['Start_Latitude'], df_filtered['Start_Longitude'],
    df_filtered['End_Latitude'], df_filtered['End_Longitude']
)

# Tạo độ cao bay (nếu chưa có)
np.random.seed(0)
df_filtered['Max_Altitude_m'] = np.random.randint(2000, 5000, len(df_filtered))
df_filtered['Min_Altitude_m'] = np.random.randint(500, 1999, len(df_filtered))


In [ ]:
plt.figure(figsize=(12, 6))
for species in species_list:
    sp_data = df_filtered[df_filtered['Species'] == species]
    regions = sp_data['Region'].unique()
    for region in regions:
        reg_data = sp_data[sp_data['Region'] == region]
        for _, row in reg_data.iterrows():
            plt.plot([row['Start_Longitude'], row['End_Longitude']],
                     [row['Start_Latitude'], row['End_Latitude']],
                     marker='o', label=f'{species} - {region}', alpha=0.6)

# Chỉ hiển thị nhãn 1 lần
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('Lộ trình di cư theo loài và vùng')
plt.xlabel('Kinh độ')
plt.ylabel('Vĩ độ')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
df_long = pd.melt(df_filtered,
                  id_vars=['Species', 'Region'],
                  value_vars=['Min_Altitude_m', 'Max_Altitude_m'],
                  var_name='Altitude_Type',
                  value_name='Altitude')

sns.barplot(data=df_long, x='Species', y='Altitude', hue='Region', ci=None, estimator=np.mean)
plt.title('Độ cao bay trung bình theo loài và vùng (gộp Min & Max)')
plt.ylabel('Độ cao (m)')
plt.xlabel('Loài chim')
plt.legend(title='Vùng')
plt.tight_layout()
plt.show()



In [ ]:

plt.figure(figsize=(10, 6))
sns.boxplot(data=df_filtered, x='Species', y='Flight_Distance_km', hue='Region')
plt.title('Khoảng cách di cư theo loài và vùng')
plt.ylabel('Khoảng cách (km)')
plt.xlabel('Loài chim')
plt.legend(title='Vùng')
plt.tight_layout()
plt.show()



1. Độ cao di cư trung bình

(Biểu đồ đầu tiên)

    Giữa các loài:

        Hawk và Warbler có độ cao bay tương đương, dao động khoảng 2350–2400m.

        Eagle có độ cao di cư hơi thấp hơn một chút so với hai loài kia.

    Giữa các vùng:

        Sự khác biệt rất nhỏ, chứng tỏ vùng địa lý không ảnh hưởng quá nhiều đến độ cao di cư.

        Tuy nhiên, Eagle ở North America bay ở độ cao thấp hơn rõ rệt so với Eagle ở Africa hoặc Asia.

2. Khoảng cách di cư

(Biểu đồ thứ hai)

    Giữa các loài:

        Cả 3 loài có khoảng cách di cư trung bình tương đối giống nhau, nhưng Eagle có xu hướng di cư xa hơn.

        Warbler và Hawk có phân bố khoảng cách đều hơn, ít chênh lệch giữa các vùng.

    Giữa các vùng:

        Có sự khác biệt rõ ràng giữa các vùng:

            Africa, Asia, và South America có khoảng cách di cư xa hơn.

            Europe và North America có khoảng cách ngắn hơn.

            Eagle ở Asia là nhóm có khoảng cách di cư xa nhất.

3. Lộ trình di cư (hướng đi)

(Biểu đồ thứ ba)

    Warbler: di cư theo hướng Đông Nam hoặc Nam, đặc biệt rõ nét ở châu Âu và châu Á.

    Hawk: lộ trình phân tán, nhưng nhiều cá thể di cư về hướng Nam và Tây Nam.

    Eagle: di chuyển theo hướng Tây hoặc Tây Nam, khá khác biệt so với 2 loài kia.
    

# 4. Phân tích ảnh hưởng môi trường đến vấn đề di cư của các loài chim

Mục tiêu: Đánh giá tác động của điều kiện thời tiết và môi trường đến hành trình di cư của các loài chim.
1. Hồi quy đa biến: Ảnh hưởng của thời tiết đến tốc độ di cư và gián đoạn

Mục tiêu: Kiểm tra xem các yếu tố thời tiết (Temperature_C, Wind_Speed_kmph, Humidity_%) ảnh hưởng thế nào đến Average_Speed_kmph (tốc độ trung bình) và Migration_Interrupted (gián đoạn di cư).

In [ ]:
# Chọn các cột liên quan
features = ['Temperature_C', 'Wind_Speed_kmph', 'Humidity_%']
target = 'Average_Speed_kmph'

# Loại bỏ các giá trị thiếu
regression_df = df[features + [target]].dropna()

# Chia dữ liệu
X = regression_df[features]
y = regression_df[target]

# Xây dựng mô hình hồi quy
model = LinearRegression()
model.fit(X, y)

# In hệ số hồi quy và R^2
print("Hệ số hồi quy:")
for feature, coef in zip(features, model.coef_):
    print(f"{feature}: {coef:.4f}")
print(f"Hệ số chặn (intercept): {model.intercept_:.4f}")
print(f"R^2 score: {r2_score(y, model.predict(X)):.4f}")

# Vẽ biểu đồ scatter để kiểm tra mối quan hệ
plt.figure(figsize=(12, 4))
for i, feature in enumerate(features):
    plt.subplot(1, 3, i+1)
    sns.scatterplot(x=regression_df[feature], y=regression_df[target])
    plt.title(f"{feature} vs {target}")
plt.tight_layout()
plt.show()

1. Hồi quy đa biến cho Average_Speed_kmph

    Hệ số hồi quy:
        Temperature_C: 0.0000 (gần như không có ảnh hưởng đáng kể). Điều này cho thấy nhiệt độ không có mối quan hệ tuyến tính rõ ràng với tốc độ di cư trung bình trong phạm vi dữ liệu này. Biểu đồ scatter cho Temperature_C vs Average_Speed_kmph cũng cho thấy phân bố dữ liệu khá đồng đều, không có xu hướng rõ rệt.
        Wind_Speed_kmph: -0.0067 (hệ số âm nhỏ). Tốc độ gió có ảnh hưởng nhẹ tiêu cực đến tốc độ di cư, nghĩa là gió mạnh hơn có thể làm giảm tốc độ bay trung bình, nhưng mức độ ảnh hưởng rất thấp. Biểu đồ scatter cho thấy dữ liệu phân bố rộng, với một số điểm ngoại lai ở tốc độ gió cao.
        Humidity_%: -0.0009 (hệ số âm rất nhỏ). Độ ẩm cũng có ảnh hưởng tiêu cực nhẹ, nhưng gần như không đáng kể. Biểu đồ scatter cho Humidity_% vs Average_Speed_kmph cho thấy sự phân bố ngẫu nhiên, không có xu hướng rõ ràng.
    Hệ số chặn (Intercept): 49.9728 (tốc độ trung bình cơ bản khi các yếu tố khác bằng 0).
    R² score: 0.0000 (rất thấp). Mô hình hồi quy này không giải thích được sự biến thiên của Average_Speed_kmph, cho thấy các yếu tố thời tiết (Temperature_C, Wind_Speed_kmph, Humidity_%) không phải là các yếu tố chính ảnh hưởng đến tốc độ di cư trong dữ liệu này. Có thể cần thêm các yếu tố khác (như loài chim, độ cao bay, hoặc điều kiện gió theo hướng) để cải thiện mô hình.

b. Hồi quy logistic cho Migration_Interrupted

Vì Migration_Interrupted là biến nhị phân (Yes/No), chúng ta sử dụng hồi quy logistic để kiểm tra tác động của Temperature_C, Wind_Speed_kmph, và Humidity_%.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Chuyển Migration_Interrupted thành nhị phân (Yes=1, No=0)
df['Migration_Interrupted_Binary'] = df['Migration_Interrupted'].map({'Yes': 1, 'No': 0})

# Chọn các cột liên quan
features = ['Temperature_C', 'Wind_Speed_kmph', 'Humidity_%']
target = 'Migration_Interrupted_Binary'

# Loại bỏ các giá trị thiếu
regression_df = df[features + [target]].dropna()

# Chia dữ liệu
X = regression_df[features]
y = regression_df[target]

# Xây dựng mô hình hồi quy logistic
model = LogisticRegression()
model.fit(X, y)

# In hệ số hồi quy
print("Hệ số hồi quy logistic:")
for feature, coef in zip(features, model.coef_[0]):
    print(f"{feature}: {coef:.4f}")
print(f"Hệ số chặn (intercept): {model.intercept_[0]:.4f}")

# In báo cáo phân loại
print("\nBáo cáo phân loại:")
print(classification_report(y, model.predict(X)))

Precision: 0.50 (lớp 0), 0.51 (lớp 1). Độ chính xác dự đoán cho cả hai lớp (No và Yes) đều thấp, cho thấy mô hình không phân biệt tốt giữa các trường hợp gián đoạn và không gián đoạn.
        Recall: 0.32 (lớp 0), 0.69 (lớp 1). Mô hình tốt hơn trong việc phát hiện các trường hợp gián đoạn (Yes), nhưng bỏ sót nhiều trường hợp không gián đoạn (No).
        F1-score: 0.39 (lớp 0), 0.58 (lớp 1). Điểm F1 trung bình, cho thấy hiệu suất tổng thể của mô hình không cao.
        Accuracy: 0.50 (50%). Mô hình chỉ nhỉnh hơn đoán ngẫu nhiên (50%), cho thấy nó không có khả năng dự đoán đáng tin cậy.
        Macro avg và weighted avg: Các giá trị F1 (0.49) và accuracy (0.50) đều thấp, khẳng định mô hình chưa hiệu quả.

Nhận xét: Mô hình hồi quy logistic cũng cho thấy các yếu tố thời tiết (Temperature_C, Wind_Speed_kmph, Humidity_%) có ảnh hưởng rất yếu đến Migration_Interrupted.

3. Nhận xét tổng quát

    Hạn chế của mô hình: Cả hai mô hình (hồi quy tuyến tính và logistic) đều có hiệu suất kém (R² = 0.0000, accuracy = 0.50), cho thấy các yếu tố thời tiết đơn lẻ (Temperature_C, Wind_Speed_kmph, Humidity_%) không đủ để giải thích sự biến thiên của Average_Speed_kmph hoặc Migration_Interrupted. Điều này có thể là do:
        Dữ liệu tổng hợp ngẫu nhiên không phản ánh các mối quan hệ thực tế giữa các biến.
        Thiếu các biến quan trọng như Weather_Condition (dạng phân loại), Max_Altitude_m, hoặc Flight_Duration_hours, vốn có thể ảnh hưởng mạnh hơn.
    Biểu đồ scatter: Các biểu đồ cho thấy phân bố dữ liệu rất đồng đều và ngẫu nhiên, không có xu hướng rõ ràng giữa các yếu tố thời tiết và Average_Speed_kmph. Điều này củng cố giả thuyết rằng dữ liệu không chứa các mối quan hệ tuyến tính mạnh.
    Đề xuất cải thiện:
        Thêm Weather_Condition (sau khi mã hóa one-hot) vào mô hình để kiểm tra tác động của các điều kiện cụ thể như Stormy hoặc Foggy.
        Sử dụng mô hình phi tuyến (như Random Forest hoặc SVM) để bắt các tương tác phức tạp giữa các yếu tố.
        Kiểm tra tương quan giữa Migration_Interrupted và các cột khác (như Interrupted_Reason, Tracking_Quality) để tìm nguyên nhân gián đoạn chính xác hơn.
        Phân tích dữ liệu theo từng loài (như Warbler hoặc Hawk) để xem liệu có sự khác biệt trong phản ứng với thời tiết không.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Mã hóa các cột phân loại
label_encoder = LabelEncoder()
df['Weather_Condition_Encoded'] = label_encoder.fit_transform(df['Weather_Condition'])
df['Food_Supply_Level_Encoded'] = label_encoder.fit_transform(df['Food_Supply_Level'])
df['Migration_Interrupted_Binary'] = df['Migration_Interrupted'].map({'Yes': 1, 'No': 0})
df['Migration_Success_Binary'] = df['Migration_Success'].map({'Successful': 1, 'Failed': 0})

# Chọn các đặc trưng (features) và mục tiêu (target)
numeric_features = ['Temperature_C', 'Wind_Speed_kmph', 'Humidity_%', 'Pressure_hPa',
                   'Visibility_km', 'Predator_Sightings', 'Weather_Condition_Encoded',
                   'Food_Supply_Level_Encoded']
target_interrupted = 'Migration_Interrupted_Binary'
target_success = 'Migration_Success_Binary'

# Tách features và target
X = df[numeric_features]
y_interrupted = df[target_interrupted]
y_success = df[target_success]

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Chia dữ liệu thành tập train và test
X_train, X_test, y_train_interrupted, y_test_interrupted = train_test_split(
    X_scaled, y_interrupted, test_size=0.2, random_state=42)
X_train, X_test, y_train_success, y_test_success = train_test_split(
    X_scaled, y_success, test_size=0.2, random_state=42)

In [ ]:
# Khởi tạo và huấn luyện mô hình Random Forest
rf_interrupted = RandomForestClassifier(n_estimators=100, random_state=42)
rf_interrupted.fit(X_train, y_train_interrupted)

# Dự đoán và đánh giá
y_pred_interrupted = rf_interrupted.predict(X_test)
print("Báo cáo phân loại cho Migration_Interrupted (Random Forest):")
print(classification_report(y_test_interrupted, y_pred_interrupted))
print("\nMa trận nhầm lẫn:")
print(confusion_matrix(y_test_interrupted, y_pred_interrupted))

# Tính độ quan trọng của các đặc trưng
feature_importance = pd.DataFrame({
    'Feature': numeric_features,
    'Importance': rf_interrupted.feature_importances_
})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

# Vẽ biểu đồ độ quan trọng
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance)
plt.title('Độ quan trọng của các đặc trưng cho Migration_Interrupted (Random Forest)')
plt.show()

In [ ]:
# Khởi tạo và huấn luyện mô hình Random Forest
rf_success = RandomForestClassifier(n_estimators=100, random_state=42)
rf_success.fit(X_train, y_train_success)

# Dự đoán và đánh giá
y_pred_success = rf_success.predict(X_test)
print("Báo cáo phân loại cho Migration_Success (Random Forest):")
print(classification_report(y_test_success, y_pred_success))
print("\nMa trận nhầm lẫn:")
print(confusion_matrix(y_test_success, y_pred_success))

# Tính độ quan trọng của các đặc trưng
feature_importance_success = pd.DataFrame({
    'Feature': numeric_features,
    'Importance': rf_success.feature_importances_
})
feature_importance_success = feature_importance_success.sort_values('Importance', ascending=False)

# Vẽ biểu đồ độ quan trọng
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_success)
plt.title('Độ quan trọng của các đặc trưng cho Migration_Success (Random Forest)')
plt.show()

In [ ]:
# Khởi tạo và huấn luyện mô hình XGBoost
xgb_interrupted = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_interrupted.fit(X_train, y_train_interrupted)

# Dự đoán và đánh giá
y_pred_interrupted_xgb = xgb_interrupted.predict(X_test)
print("Báo cáo phân loại cho Migration_Interrupted (XGBoost):")
print(classification_report(y_test_interrupted, y_pred_interrupted_xgb))
print("\nMa trận nhầm lẫn:")
print(confusion_matrix(y_test_interrupted, y_pred_interrupted_xgb))

# Tính độ quan trọng của các đặc trưng
feature_importance_xgb = pd.DataFrame({
    'Feature': numeric_features,
    'Importance': xgb_interrupted.feature_importances_
})
feature_importance_xgb = feature_importance_xgb.sort_values('Importance', ascending=False)

# Vẽ biểu đồ độ quan trọng
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_xgb)
plt.title('Độ quan trọng của các đặc trưng cho Migration_Interrupted (XGBoost)')
plt.show()

In [ ]:
# Khởi tạo và huấn luyện mô hình XGBoost
xgb_success = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_success.fit(X_train, y_train_success)

# Dự đoán và đánh giá
y_pred_success_xgb = xgb_success.predict(X_test)
print("Báo cáo phân loại cho Migration_Success (XGBoost):")
print(classification_report(y_test_success, y_pred_success_xgb))
print("\nMa trận nhầm lẫn:")
print(confusion_matrix(y_test_success, y_pred_success_xgb))

# Tính độ quan trọng của các đặc trưng
feature_importance_success_xgb = pd.DataFrame({
    'Feature': numeric_features,
    'Importance': xgb_success.feature_importances_
})
feature_importance_success_xgb = feature_importance_success_xgb.sort_values('Importance', ascending=False)

# Vẽ biểu đồ độ quan trọng
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_success_xgb)
plt.title('Độ quan trọng của các đặc trưng cho Migration_Success (XGBoost)')
plt.show()

# Phân tích chất lượng dữ liệu và công nghệ theo dõi

In [ ]:
from scipy.stats import chi2_contingency, f_oneway

In [ ]:
# Phân tích 1: Chất lượng dữ liệu (Tracking_Quality và Observation_Quality)
# So sánh Tracking_Quality với Tag_Battery_Level_% và Signal_Strength_dB
plt.figure(figsize=(12, 6))
sns.boxplot(x='Tracking_Quality', y='Tag_Battery_Level_%', data=df)
plt.title("Tag_Battery_Level_% theo Tracking_Quality")
plt.xlabel("Chất lượng theo dõi")
plt.ylabel("Mức pin (%)")
plt.show()

Phân tích chất lượng dữ liệu

    Tracking_Quality vs Tag_Battery_Level_%:
        Dự kiến: Các bản ghi với Tracking_Quality = Excellent sẽ có Tag_Battery_Level_% cao (ví dụ: trung bình 80-100%), trong khi Tracking_Quality = Poor sẽ có mức pin thấp (dưới 50%).
        Ý nghĩa: Mức pin thấp có thể làm giảm chất lượng theo dõi, dẫn đến dữ liệu không đáng tin cậy.
    Tracking_Quality vs Signal_Strength_dB:
        Dự kiến: Tracking_Quality = Excellent sẽ có Signal_Strength_dB cao hơn (ví dụ: -60 dB), trong khi Tracking_Quality = Poor có tín hiệu yếu (ví dụ: -90 dB).
        Ý nghĩa: Cường độ tín hiệu yếu có thể gây mất dữ liệu, ảnh hưởng đến độ tin cậy.
    Observation_Quality vs Tag_Battery_Level_% và Signal_Strength_dB:
        Dự kiến: Tương tự, Observation_Quality = High sẽ liên quan đến mức pin và tín hiệu cao.
        Ý nghĩa: Dữ liệu quan sát kém chất lượng (Low) có thể do thiết bị hoạt động không hiệu quả.

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='Tracking_Quality', y='Signal_Strength_dB', data=df)
plt.title("Signal_Strength_dB theo Tracking_Quality")
plt.xlabel("Chất lượng theo dõi")
plt.ylabel("Cường độ tín hiệu (dB)")
plt.show()

a. "Phân tích sự phụ thuộc của Observation_Counts vào Tag_Battery_Level_% và Signal_Strength_dB"

    Ý nghĩa: Kiểm tra xem số lần quan sát (Observation_Counts) có bị ảnh hưởng bởi mức pin (Tag_Battery_Level_%) và cường độ tín hiệu (Signal_Strength_dB) hay không. Điều này giúp đánh giá độ tin cậy của thiết bị theo dõi trong việc thu thập dữ liệu.
    Cách tiếp cận: Sử dụng biểu đồ phân tán (scatter plot) hoặc hồi quy tuyến tính để phân tích mối quan hệ.

b. "Kiểm tra sự khác biệt về Observation_Quality giữa các Tagged_By để đánh giá kỹ thuật gắn thẻ"

    Ý nghĩa: So sánh chất lượng quan sát (Observation_Quality) giữa các nhà nghiên cứu hoặc tổ chức gắn thẻ (Tagged_By) để xác định sự khác biệt trong kỹ thuật hoặc thiết bị sử dụng.
    Cách tiếp cận: Sử dụng heatmap hoặc kiểm định Chi-squared để phân tích sự phụ thuộc.

c. "Đề xuất cải thiện công nghệ: Sử dụng thiết bị nhẹ hơn, tăng tuổi thọ pin, cải thiện tín hiệu để đảm bảo chất lượng dữ liệu"

    Ý nghĩa: Dựa trên phân tích, đưa ra các giải pháp cải thiện công nghệ theo dõi, bao gồm giảm trọng lượng thiết bị (Tag_Weight_g), tăng tuổi thọ pin (Tag_Battery_Level_%), và cải thiện tín hiệu (Signal_Strength_dB).
    Liên hệ: Điều này phù hợp với tiêu chí công nghệ của BirdCast Migration Dashboard, nơi cần dữ liệu chất lượng cao để theo dõi di cư thời gian thực.

d. "Lưu ý: Dữ liệu giả lập có thể không phản ánh đúng thực tế, cần thu thập thêm dữ liệu thực tế để đánh giá chính xác hơn"

    Ý nghĩa: Nhận thức rằng dữ liệu hiện tại (10,000 bản ghi giả lập) có thể không phản ánh đầy đủ các điều kiện thực tế (ví dụ: hiệu suất thiết bị trong môi trường tự nhiên). Cần bổ sung dữ liệu thực tế để cải thiện độ chính xác của phân tích

In [ ]:
# So sánh Observation_Quality với Tag_Battery_Level_% và Signal_Strength_dB
plt.figure(figsize=(12, 6))
sns.boxplot(x='Observation_Quality', y='Tag_Battery_Level_%', data=df)
plt.title("Tag_Battery_Level_% theo Observation_Quality")
plt.xlabel("Chất lượng quan sát")
plt.ylabel("Mức pin (%)")
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='Observation_Quality', y='Signal_Strength_dB', data=df)
plt.title("Signal_Strength_dB theo Observation_Quality")
plt.xlabel("Chất lượng quan sát")
plt.ylabel("Cường độ tín hiệu (dB)")
plt.show()

Kết luận từ biểu đồ:

    Tracking_Quality khác biệt đáng kể giữa các nhà nghiên cứu, với Researcher_A và Researcher_D đạt chất lượng cao, trong khi Researcher_E có chất lượng thấp nhất.
    Sự không đồng đều này có thể do kỹ thuật gắn thẻ, loại thiết bị (Tag_Type), hoặc điều kiện triển khai (Signal_Strength_dB).

Liên hệ với phân tích trước:

    Kết quả này bổ sung bằng chứng rằng Tagged_By ảnh hưởng đến chất lượng dữ liệu, tương tự như phân tích Observation_Quality.
    Signal_Strength_dB (từ biểu đồ trước) cũng có thể giải thích sự khác biệt: Researcher_E có thể có tín hiệu yếu hơn, dẫn đến Tracking_Quality = Fair cao.

Đề xuất:

    Chuẩn hóa kỹ thuật gắn thẻ, đặc biệt cho Researcher_E, bằng cách học hỏi từ Researcher_A.
    Đảm bảo sử dụng thiết bị GPS và cải thiện tín hiệu (Signal_Strength_dB > -60 dB) cho tất cả nhà nghiên cứu.
    Tích hợp dữ liệu từ Researcher_A và Researcher_D vào BirdCast Migration Dashboard để đảm bảo chất lượng cao.
    Thu thập dữ liệu thực tế để xác nhận kết quả, như ghi chú trước đã đề cập.